# Setup

In [1]:
#@title Load Libraries
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


import nltk
import numpy as np
import random
import string
import torch
import re
import torch.nn as nn
import torch.nn.functional as F
import bs4 as bs
import urllib.request
nltk.download('stopwords')
import spacy
!pip install wmd
import wmd
!python -m spacy download en_core_web_lg
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()
!pip install rouge/requirements.txt
!pip install rouge-score
!pip install sumy
!pip install sentencepiece
!pip install language_tool_python
import sumy
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
nltk.download('punkt')
!pip install transformers
!pip install pytorch_lightning==0.7.5
# corpus = nltk.sent_tokenize(text)
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
from transformers import pipeline
from transformers import AlbertTokenizer, AlbertForPreTraining
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
albert_model = AlbertForPreTraining.from_pretrained('albert-base-v2', return_dict=True)
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pdb
import copy
!pip install transformers==2.9.0
!pip install pytorch-transformers
from transformers import AlbertTokenizer
from transformers import AlbertForPreTraining

#____
!pip install pyrouge
from pyrouge import Rouge155
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached wmd-1.3.2.tar.gz (104 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for wmd
Failed to build wmd


  DEPRECATION: Building 'wmd' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wmd'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\wmd
      copying wmd\__init__.py -> build\lib.win-amd64-cpython-311\wmd
      running build_ext
      building 'libwmdrelax' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C

ModuleNotFoundError: No module named 'wmd'

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Read Review Dataset.
*   *reviewerID*: User ID
*   *asin*: Item ID
*   *reviewText*: Review Text




In [2]:
import pandas as pd

drive_path = "../corpus_data_downloads/data/reddit_corpus_small.xlsx"
df = pd.read_excel(drive_path)

In [4]:
# Takes a String in paragraph format and saves it to the drive as a text file, each sentence having its own line.

def saveSenBySen(paragraph, saveFileFolder, hybridName) :   #arguments: String, String, String)
  for i in range (1,15):
    outFile = saveFileFolder + str(i) + "_" + hybridName
    tempList = nltk.tokenize.sent_tokenize(paragraph)
    with open(outFile, 'w') as writefile:
      for j in range(len(tempList)):
        writefile.write(tempList[j] + '\n')

# Example filepath:  "/content/drive/MyDrive/work/Leminda_sums/Leminda_sen_ny_sen/BT" + str(i) + "_" + Leminda.txt"

# Extractor and assistant methods

Cleaning Text: stop and stem words

In [5]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import *

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

#Example for removing stopwords
word_list = ['this', 'is', 'my', 'car']
new_list = [w for w in word_list if not w in stop_words]
S = "The dancers from Denmark dance a traditional dance"
senList = S.split()
noStopList = [w for w in senList if not w in stop_words]
print(noStopList)
print()
for w in noStopList :
  print(stemmer.stem(w))

['The', 'dancers', 'Denmark', 'dance', 'traditional', 'dance']

the
dancer
denmark
danc
tradit
danc


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tanner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
import os

def save_summary(summary, original_texts, val, subdir="tfidf_conversation_summaries", prefix="tfidf_", ext=".txt"):
    """
    Save original utterances and their TF-IDF summary in a single .txt file.

    Args:
        summary (list of str): TF-IDF generated summary lines.
        original_texts (list of str): Original utterances from the conversation.
        val (str): Conversation ID or other identifier.
        subdir (str): Directory where to save the file.
        prefix (str): File name prefix.
        ext (str): File extension.
    """
    base_dir = "./corpus_algorithms_and_models/corpus_data_downloads/data/"
    output_dir = os.path.join(base_dir, subdir)
    os.makedirs(output_dir, exist_ok=True)

    filename = os.path.join(output_dir, f"{prefix}{val}{ext}")

    with open(filename, "w", encoding="utf-8") as f:
        f.write("Original Response:\n")
        for line in original_texts:
            line = line.strip()
            if line:
                f.write(line + "\n")

        f.write("\nSummary:\n")
        for line in summary:
            line = line.strip()
            if line:
                f.write(line + "\n")

    print(f"Saved summary with original to: {filename}")



def load_summary(val, subdir="quillbot_results", prefix="quill_", ext=".txt"):
    """
    Load a summary text file into a single string.

    Args:
        val (str): Identifier for the filename.
        subdir (str): Directory under data/ where the file is stored.
        prefix (str): Prefix for the file name.
        ext (str): File extension.
    
    Returns:
        str: The loaded summary content.
    """
    base_dir = "./corpus_algorithms_and_models/corpus_data_downloads/data/"
    filepath = os.path.join(base_dir, subdir, f"{prefix}{val}{ext}")
    
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No summary file found at: {filepath}")
    
    with open(filepath, 'r', encoding="utf-8") as f:
        return f.read()


Edited KL_Divergence

I'm afraid of messing with someone else's code directly, so I'm copying it here and adjusting it to fit the framework of Archive and SentenceData

In [8]:
import numpy as np
from scipy.special import rel_entr

# formular for P's divergence from Q: KL(P || Q) = – sum x in X P(x) * log(Q(x) / P(x))
# If we are attempting to approximate an unknown probability distribution, then the target probability distribution from data is P, and Q is our approximation of the distribution.

def kl_archive_divergence(arcDict, arcWordNum, senDict, senWordNum):
  sum = 0;
  if( senWordNum == 0): #Sometimes punctuation will slip, causing a new sentence that has no words this filters those out.
    return 1000
  for word in senDict:
    a = arcDict[word] / arcWordNum
    b = (senDict[word] / senWordNum)
    val = a * np.log( a / b )    #Sophie version
    sum += val
    #print("for: ", word, "\t", a," * log(", a , " / ", b, ") = \t", val, flush=True)
  sum = sum / senWordNum #normalizes the scores based on sentence length.
  return sum;

p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]
def kl_ex(p, q):
	return sum(p[i] * np.log(p[i]/q[i]) for i in range(len(p)))




For getting number of sentences with the target keyword

In [9]:
def getNumSenWithKeyword(senList, keyword):
  count = 0
  #senID = 0
  #valList = []
 # print("keyword: ", keyword, "\n")
  for sen in senList :
    #senID += 1
   # print(senID, ": ", sen.GetDict())
    if (sen.GetDict().get(keyword) != None ) :
     # valList.append(senID)
      count += 1

  #print("final count: ", count)
  #print("Sentences with word: ", valList)
  return count

'''
dataSetStr = item1
revs = df.loc[df["asin"] == dataSetStr]

extRed = 2
normRed = 1
noRed = 0


print("\n\n~~~Extractor~~~~")
arc = Archive(revs, noRed, "Percent");  #use "250" , "Percent" , or "revCount"
arc.LoadSentences();
arc.getSenKeywordCount("card") '''




'\ndataSetStr = item1\nrevs = df.loc[df["asin"] == dataSetStr]\n\nextRed = 2\nnormRed = 1\nnoRed = 0\n\n\nprint("\n\n~~~Extractor~~~~")\narc = Archive(revs, noRed, "Percent");  #use "250" , "Percent" , or "revCount"\narc.LoadSentences();\narc.getSenKeywordCount("card") '

getting TF_senWord score:

In [10]:
def GetSenWordWithMaxFrequency(senDict) :  #get the frequency of the most frequent word in sentence
  maxFrequency = 0
  for word in senDict :
    if senDict[word] > maxFrequency :
      maxFrequency = senDict[word]
  return maxFrequency

def GetTF_Score(senDict, word, maxFrequency) :
  return senDict[word] / maxFrequency






getting IDF_keyword

In [11]:
import math

def GetIDF_Word(arcNumSen, numSenWithKeyword ) :
  return math.log2(arcNumSen / numSenWithKeyword)


KL_TF_IDF for one sentence

In [12]:
def KL_TF_IDF(arcNumSen, sen, arc) :
  senScore = 0
  #print(" sentence:", sen.filt_sen, "_____\n\n")
  maxFrequency = GetSenWordWithMaxFrequency(sen.GetDict())
  if maxFrequency == 0 :
    return 0
  for word in sen.GetDict() :
    numSenWithKeyword = getNumSenWithKeyword(arc.GetSenList(), word)
    IDF = GetIDF_Word(arcNumSen, numSenWithKeyword)
    TF = GetTF_Score(sen.GetDict(), word, maxFrequency)
    senScore += (IDF * np.log(IDF / TF))

  senScore = senScore / sen.filt_numWords
  return senScore


extRed = 2
normRed = 1
noRed = 0

i = 0
for testSet in dataSetList :
  i+= 1
  revs = df.loc[df["asin"] == testSet]
'''
  print("\n\n~~~Extractor TF_IDF: dataset:", i, "~~~~")
  arc = Archive(revs, noRed, "Percent");  #use "250" , "Percent" , or "revCount"
  arc.LoadSentences()
  arc.calculate_TF_IDF()
  arc.BuildSummary_TF_IDF()
  summary = arc.GetSummary()
  #print(summary)
  saveSum(summary, str(i))


#  print("\n\n\n~_~_~_~~_~_~_~ Extractor KL original _~_~_~_~")
#  arc = Archive(revs, noRed, "Percent");  #use "250" , "Percent" , or "revCount"
#  arc.LoadSentences()
#  arc.KL_divergence()
#  arc.BuildSummary()
#  summary = arc.PrintSummaryByLines() '''



NameError: name 'dataSetList' is not defined

# **Extractor**: Classes for Archive (holds/processes all the data) and SentenceData (holds info for specific sentences)

---



In [17]:
class SentenceData: # Holds original sentence, word count, filteredDictionary, and number of source review
  def __init__(self, sen, revNum):
    self.og_sen = sen
    self.og_numWords = 0
    self.filt_sen = ""
    self.filt_numWords = 0
    self.dictionary = {} #dictionary of filtered words and their frequency in sentence
    self.inSummary = False
    self.revNum = revNum
    if self.og_sen[-1] not in string.punctuation:
      self.og_sen = self.og_sen + "."

  def GetSen(self):
    return self.og_sen

  def GetOG_NumWords(self):
    return self.og_numWords

  def GetFilt_NumWords(self):
    return self.filt_numWords

  def GetDict(self):
    return self.dictionary

  def AddToSummary(self):
    self.inSummary = True

  def InSummary(self):
    return self.inSummary

  def GetRevNum(self):
    return self.revNum

  def Filter(self): #filters original sentence thru stop/stem words, puts words in dictionary with frequency
    self.filt_sen = self.og_sen.translate(str.maketrans('', '', string.punctuation)) #removes punctuation
    stop_words = set(stopwords.words('english'))
    wordList = self.filt_sen.split()
    self.og_numWords = len(wordList)
    wordList = [w for w in wordList if not w in stop_words] #filter thru stop words
    self.filt_numWords = len(wordList)

    stemmer = PorterStemmer()
    stemList = []
    self.filt_sen = ""
    for i in range(len(wordList)): #creates dictionary based on stem words
      stemWord = stemmer.stem(wordList[i])
      stemList.append(stemWord)
      if stemWord in self.dictionary:
        self.dictionary[stemWord] += 1
      else:
        self.dictionary[stemWord] = 1
      self.filt_sen += stemWord + " "
    return self.dictionary


#____________________________________________________________________________________________________


class Archive: #gets reviews, parses by sentence, gets KL scores, builds summary. Has 3 levels of affecting redundancy
  def __init__(self, revs, redunLevel, limType):
    if revs is not None:
      self.df = revs
    self.senList = []
    self.totalWords = 0
    self.fullDictionary = {}
    self.numFilterWords = 0
    self.kl_list = []
    self.TF_IDF_list = []
    self.summary = []
    self.summaryWordNum = 0
    self.summaryLimit = 250                          #UPDATE SUMMARY LENGTH HERE
    self.kl_counter = 0
    self.minSenLength = 5
    self.maxSenLength = 29
    self.redunLevel = redunLevel
    self.doc = None
    self.revCount = 0
    self.limType = limType

  def setDoc(self, doc):
    self.doc = doc

  def SetSummaryLimit(self, numRevs):
    if self.limType == "250" :
      self.summaryLimit = 250
    if self.limType == "Percent":
      self.summaryLimit = int(len(self.senList)*.1)
    if self.limType == "revCount":
      self.summaryLimit = int(numRevs / 5) * 25

  def SetSummaryLimit_doc(self):
    self.summaryLimit = int(self.totalWords * .8)  # *1 for full match. * .8 to trim size

  def GetSummaryWordNum(self):
    return self.summaryWordNum

  def GetFullDictionary(self):
      return self.fullDictionary

  def GetWordCounts(self):
   return self.totalWords, self.numFilterWords

  def GetSenList(self):
   return self.senList

  def GetLimType(self):
    return self.limType

  def KL_divergence(self): #calcs each sentence's KL_score, appends to kl_list in respective order
    self.kl_list.clear()
    for i in range(len(self.senList)):    #len(self.senList)):
      if self.senList[i].InSummary() == False:
        self.kl_list.append(kl_archive_divergence(
                            self.fullDictionary,
                            self.numFilterWords,
                            self.senList[i].dictionary,
                            self.senList[i].filt_numWords))
      else :
        self.kl_list.append(100);   # 100 when aiming for min kl (our default), -100 when aiming for max kl
    self.kl_counter += 1

  def GetKLs(self):
    return self.kl_list

  def RemoveRedundancy(self, index):  #After sentence is added to summary, remove it's filtered word occurances from the archive dictionary
    self.senList[index].AddToSummary() #flags sentence as used
    senDict = self.senList[index].GetDict()
    for word in senDict: #nothing happens if redunLevel = 0
     if (self.redunLevel == 2) : #new idea, bigger redundancy removal by dividing the remaining frequency by the used sentence frequency 10 / 3 = 3.3333
       self.fullDictionary[word] = self.fullDictionary[word] / senDict[word]
       self.numFilterWords -= senDict[word]
     if (self.redunLevel == 1) :  #OG idea, just reduces the archive frequency by the used sentence freuqency 10 - 3 = 7
       self.fullDictionary[word] -= senDict[word]
       self.numFilterWords -= senDict[word]

  def printSenAndKL(self):
    for i in range(len(self.senList)):
      print(self.kl_list[i], "\t", self.senList[i].GetSen())


  def BuildSummary(self):
    if self.limType == "Percent" :
      measure = "sentences"
    else :
      measure = "words"
    print("Building Summary. \tlimitType =", self.limType, " summary limit = ", self.summaryLimit, measure, "\n")
    counter = 0
    while (counter < self.summaryLimit):
      min_val = min(self.kl_list)
      max_index = self.kl_list.index(min_val)
      if (min_val == 100): #breaks while loop if we're out of sentences
        break
      if (self.senList[max_index].GetOG_NumWords() >= self.minSenLength
          and self.senList[max_index].GetOG_NumWords() <= self.maxSenLength) :   #only accepts 5 <= Sentence Length <= 29
        if (len(self.summary) > 0                                           #ensures that the next check doesnt go out of range.
            and self.senList[max_index].GetSen() == self.summary[-1]) :     #ignore duplicate sentences
          dummy = 0
        else:
          self.summary.append(self.senList[max_index].GetSen()) #add sentence to summary
          self.summaryWordNum += self.senList[max_index].GetOG_NumWords()    #add to total word count
          #print("sentence added: ", self.summary[-1])
      self.RemoveRedundancy(max_index)
      self.KL_divergence()
      if (self.limType == "Percent"):
        counter = len(self.summary)
      else :
        counter = self.summaryWordNum









  def BuildSummary_TF_IDF(self):
    if self.limType == "Percent" :
      measure = "sentences"
    else :
      measure = "words"
    print("Building Summary_TF_IDF. \tlimitType =", self.limType, " summary limit = ", self.summaryLimit, measure, "\n")
    counter = 0
    while (counter < self.summaryLimit):
      top_TF_IDF_score = min(self.TF_IDF_list)
      score_index = self.TF_IDF_list.index(top_TF_IDF_score)
      if (top_TF_IDF_score == 1000000): #breaks while loop if we're out of sentences
        break
      if (self.senList[score_index].GetOG_NumWords() >= self.minSenLength
          and self.senList[score_index].GetOG_NumWords() <= self.maxSenLength) :   # only accepts 5 <= Sentence Length <= 29
        if (len(self.summary) > 0                                                # ensures that the next check doesnt go out of range.
            and self.senList[score_index].GetSen() == self.summary[-1]) :          # ignore duplicate sentences
          dummy = 0
        else:
          self.summary.append(self.senList[score_index].GetSen()) #add sentence to summary
          self.summaryWordNum += self.senList[score_index].GetOG_NumWords()    #add to total word count
          #print("sentence added: ", self.summary[-1])
      self.TF_IDF_list[score_index] = 1000000   #makes it impossible to pick this sentence again
      if (self.limType == "Percent"):
        counter = len(self.summary)
      else :
        counter = self.summaryWordNum








  def PrintSummaryAsParagraph(self):
    return ' '.join(self.summary)


  def PrintSummaryByLines(self):
    print("Here is the summary by lines: , \t ", self.summaryWordNum, " words")
    print()
    for sen in self.summary:
      print(sen)
    return self.summary

  def GetSummary(self):
    return self.summary

  def LoadSentences(self):
    self.totalWords = 0;
    revCounter = 0
    #print("data", self.df)
    for i in self.df.index:

      rev = self.df.at[i, "text"] #parses out review info
      tempList = nltk.tokenize.sent_tokenize(rev) #splits review into sentences
      for j in range(len(tempList)):
        sen = SentenceData(tempList[j], revCounter) #creates SentenceData
        self.senList.append(sen) #adds Sentence data to list
        self.totalWords += len(tempList[j].split())
        senDict = self.senList[-1].Filter() #returns SentenceData's filtered dictionary

        for word in senDict: #add sentence filter dictionary to archive filter dictionary
          if word in self.fullDictionary:
            self.fullDictionary[word] += senDict[word]
          else:
            self.fullDictionary[word] = senDict[word]
          self.numFilterWords += senDict[word]
      revCounter += 1
    self.SetSummaryLimit(revCounter)
    self.revCount = revCounter


  def LoadSentences_doc(self, doc):  #Input is just one line/paragraph.
    self.totalWords = 0;
    revCounter = 0
    self.doc = doc
    tempList = nltk.tokenize.sent_tokenize(self.doc) #splits review into sentences
    for j in range(len(tempList)):
      sen = SentenceData(tempList[j], revCounter) #creates SentenceData
      self.senList.append(sen) #adds Sentence data to list
      self.totalWords += len(tempList[j].split())
      senDict = self.senList[-1].Filter() #returns SentenceData's filtered dictionar
      for word in senDict: #add sentence filter dictionary to archive filter dictionary
        if word in self.fullDictionary:
          self.fullDictionary[word] += senDict[word]
        else:
          self.fullDictionary[word] = senDict[word]
        self.numFilterWords += senDict[word]
    revCounter += 1
    self.SetSummaryLimit_doc()

  def getSenKeywordCount(self, keyword) :
    return getNumSenWithKeyword(self.senList, keyword)

  def calculate_TF_IDF(self) :
    senID = 0
    print("Calculating TF_IDF senScores: ")
    for sen in self.senList :
      senScore = KL_TF_IDF( len(self.senList), sen, self)
      #print(senID, ": ", "senScore: ", senScore)
      senID += 1
      self.TF_IDF_list.append(senScore)

#______________________________________________________________________________________________
#_______________________________________________________________________________________________


Using the Extractor:

In [22]:
extRed = 2
normRed = 1
noRed = 0
nGrams = 2  # for n-gram redundancy checking

df = df[df["text"].notnull()]


dataSetList = df["conversation_id"].dropna().unique().tolist()

for i, convo_id in enumerate(dataSetList, start=1):
    print(f"\n\n~~~ Extractor TF_IDF: conversation {i} (ID: {convo_id}) ~~~")

    revs = df.loc[df["conversation_id"] == convo_id]
    if revs.empty:
        print(f"Warning: No utterances found for conversation ID {convo_id}")
        continue

    arc = Archive(revs, noRed, "Percent")  # adjust "Percent" as needed
    arc.LoadSentences()
    arc.calculate_TF_IDF()
    arc.BuildSummary_TF_IDF()

    summary = arc.PrintSummaryByLines()  # returns list of summary sentences

    original_texts = revs["text"].dropna().astype(str).tolist()
    save_summary(summary, original_texts, val=convo_id)





~~~ Extractor TF_IDF: conversation 1 (ID: 9c716m) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  3 sentences 

Here is the summary by lines: , 	  40  words

&gt; if you think I talked to any of my friends in the manner he posted.
Send his [candid pics without makeup](https://koreaboo-cdn.storage.googleapis.com/2017/10/GDragon-Beard-01.jpg).
"Bruh", you're stupid if you think I talked to any of my friends in the manner he posted.
Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9c716m.txt


~~~ Extractor TF_IDF: conversation 2 (ID: 9c8amk) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  2 sentences 

Here is the summary by lines: , 	  18  words

Wth they have air conditioning on bus stops.
Well, some of our bus interchange have air con too.
Saved summary with original to: ./corpus_algorithms_and_models/corpus

C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))


Building Summary_TF_IDF. 	limitType = Percent  summary limit =  20 sentences 

Here is the summary by lines: , 	  193  words

I don't do it because I want something.
I say "are you sure you want to do this?"
I do it because I enjoy doing it.
I don't think it was a miscommunication.
I really like blowjobs, who doesn't?
She says "I don't mind" and keeps going.
And she says "I don't mind" and keeps going.
And I say "I'm really enjoying this blowjob" so she keeps going.
Don't get me wrong - I love eating her out.
Should I not have said that?
You said "she's at fault, but not to blame".
She starts giving me a blowjob...
But I don't get a lot of blowjobs in return.
I have a feeling that this might be the reason the responses are what they are.
That is exactly how I see it.
Like i straight up said, talk.
My partner and I have had years of this.
I was abused, the people who did that are only to blame for their actions.
And obviously, those women should have communicated better.
She also told m

C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))


Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  2 sentences 

Here is the summary by lines: , 	  19  words

Sorry for deleting that comment.
Would you add something from your side along with the Code for the Pika?
Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9c8e46.txt


~~~ Extractor TF_IDF: conversation 816 (ID: 9cg4q3) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  2 sentences 

Here is the summary by lines: , 	  20  words

I'm currently breeding for other trades, but I'll start working on those too :)
I have yours done so just lmk.
Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9cg4q3.txt


~~~ Extractor TF_IDF: conversation 817 (ID: 9cgn08) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary

...

C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))


Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  0 sentences 

Here is the summary by lines: , 	  0  words

Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9hfwy4.txt


~~~ Extractor TF_IDF: conversation 1680 (ID: 9hhjvg) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  0 sentences 

Here is the summary by lines: , 	  0  words

Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9hhjvg.txt


~~~ Extractor TF_IDF: conversation 1681 (ID: 9hmpxi) ~~~
Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  6 sentences 

Here is the summary by lines: , 	  64  words

Two parts of our brains disagree.
Evolution says: This is a result of our evolution from knuckle walking apes.
Evolution says: This was a direct ro

C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))


Calculating TF_IDF senScores: 
Building Summary_TF_IDF. 	limitType = Percent  summary limit =  0 sentences 

Here is the summary by lines: , 	  0  words

Saved summary with original to: ./corpus_algorithms_and_models/corpus_data_downloads/data/tfidf_conversation_summaries\tfidf_9i81e6.txt


C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: divide by zero encountered in log
  senScore += (IDF * np.log(IDF / TF))
C:\Users\Tanner\AppData\Local\Temp\ipykernel_20592\221282513.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  senScore += (IDF * np.log(IDF / TF))


# Abstractor

**BART **(Abstractive summary)
import transformer and Bart before run the code below (at the very top of this notebook)

There are different hypo-patameters avaliable to use (max_length, min_length, etc.) Learn more here: https://huggingface.co/docs/transformers/model_doc/bart

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def run_bart(asin_text):

    inputs = tokenizer.batch_encode_plus([asin_text],return_tensors = 'pt', max_length = 1024)
    summary_ids = model.generate(inputs['input_ids'], max_length = 400, min_length = 320) #current at 400, 320 . trying 600, 520
    bart_summary = tokenizer.decode(summary_ids[0],skip_special_tokens = True)
    return bart_summary

In [ ]:
def run_bart_beam(asin_text, numBeam):

    inputs = tokenizer.batch_encode_plus([asin_text],return_tensors = 'pt', max_length = 1024)
    summary_ids = model.generate(inputs["input_ids"], num_beams=numBeam, max_length=400, min_length = 320)
    bart_summary = tokenizer.decode(summary_ids[0],skip_special_tokens = True)


    return bart_summary

def run_bart_multinomial_beam(asin_text, numBeam):

    inputs = tokenizer.batch_encode_plus([asin_text],return_tensors = 'pt', max_length = 1024)
    summary_ids = model.generate(inputs["input_ids"], do_sample=True, num_beams=numBeam, max_length=400, min_length = 320)
    bart_summary = tokenizer.decode(summary_ids[0],skip_special_tokens = True)


    return bart_summary


# Formatting Extractor and Abstractor Results

In [ ]:
def shorten_extract(senList, targetLen):
  maxLen = 500
  if targetLen > maxLen:
    targetLen = maxLen
  else:
    targetLen = targetLen - 1 #shorten gives 1 sentence over the targetLen. For summaries with 250 word input from extractor, the abstractor will generate 1 completely new sentence. subtract 1 to counter this.

  wordCount = 0
  wordCountList = []
  tooLong = True
 # print("GETTING WORD COUNT")
  for i in range(len(senList)):
    num = len(senList[i].split())
    wordCountList.append(num)
    wordCount += num

 # print("POPPING SENTENCES")
  while tooLong == True :
    if (wordCount - wordCountList[-1] > targetLen ):
      senList.pop(-1)
      wordCount -= wordCountList[-1]
      wordCountList.pop(-1)
    else :
      tooLong = False
  return senList, wordCount

In [ ]:
def shorten_abstract(abs, targetLen):
  if targetLen > 250:
    targetLen = 250
  else:
    targetLen = targetLen - 1 #shorten gives 1 sentence over the targetLen. For summaries with 250 word input from extractor, the abstractor will generate 1 completely new sentence. subtract 1 to counter this.

  senList = nltk.tokenize.sent_tokenize(abs)
  wordCount = 0
  wordCountList = []
  tooLong = True
  for i in range(len(senList)):
    num = len(senList[i].split())
    wordCountList.append(num)
    wordCount += num

  while tooLong == True :
    if (wordCount - wordCountList[-1] > targetLen ):
      senList.pop(-1)
      wordCount -= wordCountList[-1]
      wordCountList.pop(-1)
    else :
      tooLong = False
  return ' '.join(senList), wordCount




# **Hybrid Summarizer**

In [ ]:
class HybridData: # Holds original sentence, word count, filteredDictionary, and number of source review
  def __init__(self, ext250Sum = 0, extPercentSum= 0, ext250WordCount = 0, extPercentWordCount = 0):
    self.ext250 = ext250Sum
    self.extPercent = extPercentSum
    self.ext250WordCount = ext250WordCount
    self.extPercentWordCount = extPercentWordCount
    self.numBeams = 10

  def Set250(self, summary, wordCount):
    self.ext250 = summary
    self.ext250WordCount = wordCount

  def SetPercent(self, summary, wordCount):
    self.extPercent = summary
    self.extPercent = wordCount

  def Get250Paragraph(self) :
    return ' '.join(self.ext250)

  def GetPercentParagraph(self) :
    return ' '.join(self.extPercent)

  def Generate250(self, revs) :
    noRed = 0
    normRed = 1
    extRed = 2
    arc = Archive(revs, extRed, "250");  #use "250" , "Percent" , or "revCount"
    arc.LoadSentences();
    arc.KL_divergence()
    print()
    arc.BuildSummary()
    self.ext250WordCount = arc.GetSummaryWordNum()
    self.ext250 = arc.PrintSummaryByLines()


  def GeneratePercent(self, revs) :
    noRed = 0
    normRed = 1
    extRed = 2
    arc = Archive(revs, noRed, "Percent");
    arc.LoadSentences();
    #arc.KL_divergence()
    arc.calculate_TF_IDF()
    print()
    #arc.BuildSummary()
    arc.BuildSummary_TF_IDF()
    self.extPercentWordCount = arc.GetSummaryWordNum()
    self.extPercent = arc.PrintSummaryByLines()




  def RunExtractor(self, itemID):
    itemID = itemID - 1

    item1  = "B0143IIP4W" # (SD card),            28 reviews, 125     1             76
    item2  = "B00WWBCQEI" #(camera memory card),  31 reviews, 150     2             144
    item3  = "B01BGTG41W" #(flash drive)       ,  35 reviews, 175     4             132
    item4  = "B00006B81E" #(powerstrip),          36 reviews, 175     6             171
    item5  = "B00004ZCJJ" #(62mm protect filter), 53 reviews, 250     5             176
    item6  = "B000BQ7GW8" #(2gb SD card)          43 reviews, 200     7             213
    item7  = "B011M8YACM" #(hard drive adapter),  35 reviews, 175     3             271
    item8  = "B003L1ZYYW" #(HDMI cables)          80 reviews,         14            276
    item9  = "B009D79VH4" #(card reader)          71 reviews,         12            325
    item10 = "B001TH7GUU" #(USB extension)        87 reviews,         15            364
    item11 = "B0000BVYT3" #(Ethernet switch)      70 reviews, 350     9             483
    item12 = "B0006BB9MG" #(cable ties)           88 reviews, 425     10            500
    item13 = "B000HPV3RW" #(surge protector)      65 reviews,         11            568
    item14 = "B00KFD0SEA" #ethernet switch)       75 reviews,         13            1000

    dataSetList = ["B0143IIP4W", "B00WWBCQEI", "B01BGTG41W", "B00006B81E", "B00004ZCJJ",
                   "B000BQ7GW8", "B011M8YACM", "B003L1ZYYW", "B009D79VH4", "B001TH7GUU",
                   "B0000BVYT3", "B0006BB9MG", "B000HPV3RW", "B00KFD0SEA"]

    revs = df.loc[df["asin"] == dataSetList[itemID]]

    print("\n\nGenerating extPercent Summary for dataSet: ", itemID + 1, ":\t", dataSetList[itemID])
    self.GeneratePercent(revs)
    #self.SaveSummary(self.extPercent, str(itemID + 1), "extPercent.noRed")

  def LingEvals(self):
    self.lingList250 = []
    nGrams = 2
    redunScore = -1 * (1 - uniqueNgramRatio(self.Get250Paragraph(), nGrams))

    self.lingList250.append(0)                                                  #get_coherence_score2(self.ext250))
    self.lingList250.append(get_focus_score(self.ext250))
    self.lingList250.append(redunScore)                                         #get_redundency_score(self.ext250))
    self.lingList250.append(grammarScore(self.ext250))
    self.lingList250.append(sum(self.lingList250))

    self.lingListPercent = []
    redunScore =  -1 * (1 - uniqueNgramRatio(self.GetPercentParagraph(), nGrams))
    self.lingListPercent.append(0)                                              #get_coherence_score2(self.extPercent))
    self.lingListPercent.append(get_focus_score(self.extPercent))
    self.lingListPercent.append(redunScore)                                     #get_redundency_score(self.extPercent))
    self.lingListPercent.append(grammarScore(self.extPercent))
    self.lingListPercent.append(sum(self.lingListPercent))

  def PrintLingEvals(self):
    print("ext250 LingEvals: ")
    print("grammer    : \t", self.lingList250[3])
    print("coherence  : \t", self.lingList250[0])
    print("redundancy : \t", self.lingList250[2])
    print("focus      : \t", self.lingList250[1])
    print("Readability: \t", self.lingList250[4])

    print("\n\nextPercent LingEvals: ")
    print("grammer    : \t", self.lingListPercent[3])
    print("coherence  : \t", self.lingListPercent[0])
    print("redundancy : \t", self.lingListPercent[2])
    print("focus      : \t", self.lingListPercent[1])
    print("Readability: \t", self.lingListPercent[4])

  def shorten_abstract(self, senList, targetLen):
    if targetLen > 250:
      targetLen = 250
    else:
      targetLen = targetLen - 1 #shorten gives 1 sentence over the targetLen. For summaries with 250 word input from extractor, the abstractor will generate 1 completely new sentence. subtract 1 to counter this.

    wordCount = 0
    wordCountList = []
    tooLong = True
    for i in range(len(senList)):
      num = len(senList[i].split())
      wordCountList.append(num)
      wordCount += num

    while tooLong == True :
      if (wordCount - wordCountList[-1] > targetLen):
        senList.pop(-1)
        wordCount -= wordCountList[-1]
        wordCountList.pop(-1)
      else :
        tooLong = False
    return senList, wordCount

  def RunAbstractor(self):
    numBeams = 10
    wordNum = 0
    if (self.lingList250[4] > self.lingListPercent[4]) :
      self.Run250ToAbstractor()
      wordNum = self.ext250WordCount
    else :
      self.RunPercentToAbstractor()
      wordNum = self.extPercentWordCount

    for sen in self.hybridSum:
      print(sen)
    self.resizedHybridSum, self.resizedHybridWordCount = self.shorten_abstract(self.hybridSum, wordNum)
    return self.resizedHybridSum

  def Run250ToAbstractor(self):
    print("\nAbstractor Using ext250")
    self.hybridSum = nltk.tokenize.sent_tokenize(run_bart_beam(self.Get250Paragraph(), self.numBeams))

  def RunPercentToAbstractor(self):
    print("\nAbstractor Using extPercent")
    self.hybridSum = nltk.tokenize.sent_tokenize(run_bart_beam(self.GetPercentParagraph(), self.numBeams))

  def PrintHybridSum(self):
    for sen in self.resizedHybridSum:
      print(sen)

  def SaveSummary(self, summary, setVal, sumType): #Takes summary (divided line by line) and saves it
    filename = "/content/drive/MyDrive/work/BartTest_SummaryFiles/BT" + setVal + "/" + "BT" + setVal + "_a" + sumType + ".txt"
    with open(filename, 'w') as writefile:
      for j in range(len(summary)):
        writefile.write(summary[j] + "\n")
    print("saved: ", filename)


  def RunHybridDecider(self, itemID):
    self.RunExtractor(itemID)
    self.LingEvals()
    self.PrintLingEvals()
    self.RunAbstractor()
    self.SaveSummary(self.resizedHybridSum, str(itemID), "zhybridSum")
    print()
    print()
    print("Final Result")
    self.PrintHybridSum()


  '''def RunBothHybrids(self, itemID):
    self.RunExtractor(itemID)
    self.LingEvals()
    self.PrintLingEvals()

    self.Run250ToAbstractor()
    self.resizedHybridSum, self.resizedHybridWordCount = self.shorten_abstract(self.hybridSum, self.ext250WordCount)
    self.SaveSummary(self.resizedHybridSum, str(itemID), "n250ToAbs")
    print("\nprinting 250ToAbs")
    self.PrintHybridSum()
    print("\n\n")

    self.RunPercentToAbstractor()
    self.resizedHybridSum, self.resizedHybridWordCount = self.shorten_abstract(self.hybridSum, self.extPercentWordCount)
    self.SaveSummary(self.resizedHybridSum, str(itemID), "nPercentToAbs")
    self.PrintHybridSum()
    if (self.lingList250[4] > self.lingListPercent[4]) :
      print("\nChosen hybrid is ext250")
    else:
      print("\nChosen hybrid is extPercent")'''



  def RunHybrid(self, itemID):
    self.RunExtractor(itemID)
    self.RunPercentToAbstractor()
    self.resizedHybridSum, self.resizedWordCount = self.shorten_abstract(self.hybridSum, self.extPercentWordCount)
    #self.SaveSummary(self.resizedHybridSum, str(itemID), "zhybrid.xtRed")
    print()
    print()
    print("Final Result, using", self.resizedWordCount, "words:\n")
    self.PrintHybridSum()



hybrid = HybridData()

'''
  print("\n\n~~~Extractor TF_IDF: dataset:", i, "~~~~")
  arc = Archive(revs, noRed, "Percent");  #use "250" , "Percent" , or "revCount"
  arc.LoadSentences()
  arc.calculate_TF_IDF()
  arc.BuildSummary_TF_IDF()
  summary = arc.GetSummary()
  #print(summary)
  saveSum(summary, str(i))
'''


for i in range(14):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Dataset", i + 1, "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  hybrid.RunHybrid(i + 1)











Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Dataset 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Generating extPercent Summary for dataSet:  1 :	 B0143IIP4W
Calculating TF_IDF senScores: 

Building Summary_TF_IDF. 	limitType = Percent  summary limit =  0 sentences 

Here is the summary by lines: , 	  0  words


Abstractor Using extPercent


IndexError: list index out of range

In [ ]:
#@title Running the hybrid
import time

hybrid = HybridData()
runTimeList = []


for i in range(14):

  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Dataset", i + 1, "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  startTime = time.process_time()
  hybrid.RunHybrid(i + 1)
  endTime = time.process_time()
  runTime = endTime - startTime
  runTimeList.append(runTime)
  print("run time: ", runTime)

j = 0
print("\n\nrunTimes for KL_tfidf: ")
for val in runTimeList:
  j += 1
  print(j, ":\t", val)